In [2]:
from datetime import datetime
import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from tqdm import tqdm

import os
from time import sleep

from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By

import requests
from bs4 import BeautifulSoup

In [3]:
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [4]:
url = 'https://www.loket.com/discover?per_page=8&sorts=daz&page='

In [5]:
# get the chromedriver path
path = os.getcwd()+'/chromedriver'
options = Options()
#options.headless = True
#options.add_argument("--disable-gpu")
#options.add_argument("--no-sandbox")
#options.add_argument("enable-automation")
#options.add_argument("--disable-infobars")
#options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome(options=options)

In [6]:
# Execute the webdriver
driver.get(url)

In [7]:
soup = driver.page_source
bsobj = BeautifulSoup(soup,'html.parser')

## Get URLS

In [8]:
Urls = []

In [ ]:
for i in range (1,72):
    soup = driver.page_source
    bsobj = BeautifulSoup(soup,'html.parser')
    
    for ti in bsobj.findAll('a',{'class':'uk-link-reset body'}):
        each_url = ti['href']
        what_url = each_url
        Urls.append(what_url)
        
    sleep(10)
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//span[@class="uk-link-reset next-page"]'))).click()

In [8]:
Urls

['https://www.loket.com/event/optimalisasi-kecerdasan-anak-strategi-pembinaan-stimulasi-yang-tepat-vzdg',
 'https://www.loket.com/event/10-indonesian-private-lessons_V999g',
 'https://www.loket.com/event/happy-asmara-mr-jono-joni-ngopibareng-pintulangit-vly',
 'https://www.loket.com/event/managing-across-generation_VEbb',
 'https://www.loket.com/event/10-indonesian-private-lessons_V9993',
 'https://www.loket.com/event/aspek-dan-perlindungan-hukum-terhadap-perkawinan-masyarakat-penganut-vbydd',
 'https://www.loket.com/event/talk8-jaf2023',
 'https://www.loket.com/event/moon-celebration-v24b',
 'https://www.loket.com/event/optimalisasi-kecerdasan-anak-strategi-pembinaan-stimulasi-yang-tepat-vzdg',
 'https://www.loket.com/event/10-indonesian-private-lessons_V999g',
 'https://www.loket.com/event/happy-asmara-mr-jono-joni-ngopibareng-pintulangit-vly',
 'https://www.loket.com/event/managing-across-generation_VEbb',
 'https://www.loket.com/event/10-indonesian-private-lessons_V9993',
 'https:/

In [9]:
df_url = pd.DataFrame(Urls, columns = ['Urls'])

In [10]:
#df_url.to_csv("url_loketdotcom_08062023.csv")

## Now move on

In [11]:
#df_url = pd.read_csv("url_loketdotcom_31052023.csv")

In [12]:
df_url

,Urls
0,https://www.loket.com/event/optimalisasi-kecer...
1,https://www.loket.com/event/10-indonesian-priv...
2,https://www.loket.com/event/happy-asmara-mr-jo...
3,https://www.loket.com/event/managing-across-ge...
4,https://www.loket.com/event/10-indonesian-priv...
...,...
481,https://www.loket.com/event/rekaman-zoom-resep...
482,https://www.loket.com/event/rekaman-zoom-resep...
483,https://www.loket.com/event/rekaman-resep-baki...
484,https://www.loket.com/event/rekaman-resep-baki...


In [13]:
def date_indo_full(start, time):
    new_date = start.lower().strip()
    world_date = new_date.replace('wib','').replace('wita','').replace('wit','').replace('jan', 'Jan').replace('feb', 'Feb').replace('mar', 'Mar').replace('apr', 'Apr').replace('mei', 'May').replace('jun', 'Jun').replace('jul', 'Jul').replace('agt', 'Aug').replace('sep', 'Sep').replace('okt', 'Oct').replace('nov', 'Nov').replace('des', 'Dec').strip()
    start_date = datetime.strptime(world_date, "%d %b %Y")
    
    time = time.lower().replace('wib','').replace('wita','').replace('wit','').strip()
    time_split = time.split(':')
    final_date = start_date.replace(hour = int(time_split[0]), minute = int(time_split[1]))
    
    return final_date

In [14]:
def loketdotcom_detail(url):
    test = requests.get(url = url, headers = header)
    bsobj = BeautifulSoup(test.content,'html.parser')
    
    print(bsobj)
    Event_name=bsobj.find('h1',{'id':'gt-event-name'}).text.strip()
    Category_event=bsobj.find_all('li',{'class':'event-detail-breadcrumbs-item'})[1].text.strip().replace('\n',' ')
    Venue = bsobj.find('div',{'class':'event-venue'}).text.strip()
    Event_time=bsobj.find('div',{'class':'event-time'}).text.strip()
    try:
        Desc = bsobj.find('ul',{'class':'uk-switcher description-panel'}).text.strip().replace('\n',' ').replace('\xa0',' ')
    except:
        Desc = None
    try:
        Price = bsobj.find('span',{'id':'sub-total-price'}).text
    except:
        Price = None
    try:
        loc = bsobj.find('div',{'class':'event-venue'}).a['href']
        lat_lon = loc.split('query=')[1]
        lat = lat_lon.split(',')[0]
        lon = lat_lon.split(',')[1]
    except:
        lat=None
        lon=None
        
    Event_date=bsobj.find('div',{'class':'event-date'}).text.strip()
    
    try:
        begin_date = Event_date.split('-')[0] + '2023'
        end_date = Event_date.split('-')[1]
    except:
        begin_date = Event_date
        end_date = Event_date
        
    Time_event = bsobj.find('div',{'class':'event-time'}).text.split('-')
    print(Time_event)
    
    begin_time = Time_event[0]
    end_time = Time_event[1]
    
    Start_date = date_indo_full(begin_date, begin_time)
    End_date = date_indo_full(end_date, end_time)
    
    data = {
        'Event Name': [Event_name],
        'Start Date': [Start_date],
        'End Date': [End_date],
        'Category Event': [Category_event],
        'Price': [Price],
        'Venue': [Venue],
        'Address': [Venue],
        'Description': [Desc],
        'Source': [url],
        'Latitude': [lat],
        'Longitude': [lon]
    }
    
    df = pd.DataFrame(data)   

    return df

In [15]:
def loketdotcom_detail_selenium(url):
    test = driver.get(url)
    soup = driver.page_source
    bsobj = BeautifulSoup(soup,'html.parser')
    
    Event_name=bsobj.find('h1',{'id':'gt-event-name'}).text.strip()
    Category_event=bsobj.find_all('li',{'class':'event-detail-breadcrumbs-item'})[1].text.strip().replace('\n',' ')
    Venue = bsobj.find('div',{'class':'event-venue'}).text.strip()
    Event_time=bsobj.find('div',{'class':'event-time'}).text.strip()
    try:
        Desc = bsobj.find('ul',{'class':'uk-switcher description-panel'}).text.strip().replace('\n',' ').replace('\xa0',' ')
    except:
        Desc = None
    try:
        Price = bsobj.find('span',{'id':'sub-total-price'}).text
    except:
        Price = None
    try:
        loc = bsobj.find('div',{'class':'event-venue'}).a['href']
        lat_lon = loc.split('query=')[1]
        lat = lat_lon.split(',')[0]
        lon = lat_lon.split(',')[1]
    except:
        lat=None
        lon=None
        
    Event_date=bsobj.find('div',{'class':'event-date'}).text.strip()
    
    try:
        begin_date = Event_date.split('-')[0] + '2023'
        end_date = Event_date.split('-')[1]
    except:
        begin_date = Event_date
        end_date = Event_date
        
    Time_event = bsobj.find('div',{'class':'event-time'}).text.split('-')
    
    
    begin_time = Time_event[0].strip()
    end_time = Time_event[1].strip()
    
    Start_date = date_indo_full(begin_date, begin_time)
    End_date = date_indo_full(end_date, end_time)
    
    data = {
        'Event Name': [Event_name],
        'Start Date': [Start_date],
        'End Date': [End_date],
        'Category Event': [Category_event],
        'Price': [Price],
        'Venue': [Venue],
        'Address': [Venue],
        'Description': [Desc],
        'Source': [url],
        'Latitude': [lat],
        'Longitude': [lon]
    }
    
    df = pd.DataFrame(data)   

    return df

In [16]:
header = {
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36',
    'referer':'https://www.loket.com/'
}

In [17]:
df_detail = pd.DataFrame()

In [18]:
for i in tqdm(range(len(df_url["Urls"]))):
    urls = df_url["Urls"][i]
    try:
        dfs = loketdotcom_detail_selenium(urls)
    except:
        print('Gagal' + ' '+ urls )
        continue
    df_detail = pd.concat([df_detail, dfs], ignore_index=True)
    print(urls)
    sleep(1)    

  0%|                                                                                          | 0/486 [00:00<?, ?it/s]

https://www.loket.com/event/optimalisasi-kecerdasan-anak-strategi-pembinaan-stimulasi-yang-tepat-vzdg


  0%|▏                                                                                 | 1/486 [00:05<47:36,  5.89s/it]

https://www.loket.com/event/10-indonesian-private-lessons_V999g


  0%|▎                                                                                 | 2/486 [00:09<38:31,  4.78s/it]

...

Gagal https://www.loket.com/event/rekaman-resep-baking-class-with-rozma-cookies-jadul_UY99U


100%|███████████████████████████████████████████████████████████████████████████████▊| 485/486 [38:02<00:04,  4.41s/it]

Gagal https://www.loket.com/event/rekaman-resep-baking-class-with-rozma-suhardi-aneka_UY999


100%|████████████████████████████████████████████████████████████████████████████████| 486/486 [38:08<00:00,  4.71s/it]

Gagal https://www.loket.com/event/rekaman-resep-baking-class-with-rozma-suhardi-brownies_UYDcU


In [ ]:
df_detail.to_csv("Loketdotcom_27September2023.csv")

In [1]:
df_detail

NameError: name 'df_detail' is not defined

In [55]:
test = requests.get(url = test_url1, headers = header)

In [56]:
bsobj = BeautifulSoup(test.content,'html.parser')

In [53]:
Event_name=bsobj.find('h1',{'id':'gt-event-name'}).text

In [39]:
Start_date=bsobj.find('div',{'class':'event-date'}).text.strip()

In [13]:
Category_event=bsobj.find_all('li',{'class':'event-detail-breadcrumbs-item'})[1].text.strip().replace('\n',' ')

'28 Mar - 29 Mar 2023'

In [62]:
Venue = bsobj.find('div',{'class':'event-venue'}).text.strip()

In [58]:
Event_time=bsobj.find('div',{'class':'event-time'}).text

In [64]:
Price = bsobj.find('span',{'id':'sub-total-price'}).text

In [65]:
Price

'0'

In [17]:
bsobj.select('ul > li')[2]

<li class="navbar-link uk-visible@m">
<a href="https://www.loket.com/pricing">Biaya</a>
</li>

In [40]:
Start_date

'28 Mar - 29 Mar 2023'

In [57]:
bsobj

<!DOCTYPE html>

<html lang="id-ID" prefix="og: http://ogp.me/ns#">
<head>
<title>ENHYP3N M4NIFESTO in DEPOK [contoh] | Loket.com</title>
<meta charset="utf-8"/><script type="text/javascript">(window.NREUM||(NREUM={})).init={privacy:{cookies_enabled:false},ajax:{deny_list:["bam.nr-data.net"]}};(window.NREUM||(NREUM={})).loader_config={xpid:"VgUPWVRVCBADUllWAgAOV10=",licenseKey:"2c15469c42",applicationID:"368451919"};;(()=>{var e,t,r={9071:(e,t,r)=>{"use strict";r.d(t,{I:()=>n});var n=0,i=navigator.userAgent.match(/Firefox[\/\s](\d+\.\d+)/);i&&(n=+i[1])},6562:(e,t,r)=>{"use strict";r.d(t,{P_:()=>g,Mt:()=>v,C5:()=>d,DL:()=>y,OP:()=>N,lF:()=>L,Yu:()=>E,Dg:()=>p,CX:()=>f,GE:()=>w,sU:()=>k});var n={};r.r(n),r.d(n,{agent:()=>T,match:()=>S,version:()=>A});var i=r(6797),o=r(909),a=r(8610);class s{constructor(e,t){try{if(!e||"object"!=typeof e)return(0,a.Z)("New setting a Configurable requires an object as input");if(!t||"object"!=typeof t)return(0,a.Z)("Setting a Configurable requires a model 